- 使用overlap： 这里不是指的滑动窗口切块的过程中有一定窗口重叠，而是召回的时候会去检索前后的文档块，llamaindex中有比较方便的实现。



In [1]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
import faiss
import os
import sys
from dotenv import load_dotenv
from pprint import pprint

In [2]:
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from openai import OpenAI
from typing import Optional, List, Mapping, Any

class OurLLM(CustomLLM):
    context_window: int = 10000
    num_output: int = 512
    model_name: str = "glm-4-plus"
    llm:str =  ""
    
    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        self.llm = OpenAI(base_url="https://open.bigmodel.cn/api/paas/v4",api_key="f97b28aa71892d6d515007ef44ee5de6.WvLXD1MsFsGklOZa")
        completion = self.llm.chat.completions.create(model="glm-4-plus",temperature=0.0,messages=[{"role": "user", "content": prompt}])
        return CompletionResponse(text=completion.choices[0].message.content)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        response = ""
        self.llm = OpenAI(base_url="https://open.bigmodel.cn/api/paas/v4",api_key="f97b28aa71892d6d515007ef44ee5de6.WvLXD1MsFsGklOZa")
        completion = self.llm.chat.completions.create(model="glm-4-plus",temperature=0.0,messages=[{"role": "user", "content": prompt}],
                                                     stream=True)
        for token in completion:
            delta = token.choices[0].delta.content
            response += delta
            yield CompletionResponse(text=response, delta=delta)

D:\python311\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
EMBED_DIMENSION=512
Settings.llm = OurLLM()
Settings.embed_model = HuggingFaceEmbedding(model_name="../../bge-small-zh-v1.5")

In [5]:
from llama_index.core.schema import Document

In [6]:
text = """蔚来萤火虫即将交付，李斌所预测的两年内新能源车渗透率超80%能否实现？

9 月 6 日消息，近日，蔚来汽车创始人、董事长李斌在蔚来九周年内部讲话及财报电话会上透露了多项重要信息，其中最引人注目的莫过于第三品牌萤火虫将于 2025 年正式交付，并预测新能源汽车的市场渗透率将在未来两年内超过 80%。

据李斌介绍，蔚来汽车将形成三个品牌矩阵，覆盖从 14 万元到 80 万元的广阔市场区间。其中，第三品牌萤火虫作为蔚来汽车布局中低端市场的重要棋子，将于 2025 年正式交付。这一举措不仅丰富了蔚来的产品线，也进一步提升了其在新能源汽车市场的竞争力。

在谈及新能源汽车市场渗透率时，李斌表示出极大的信心。他指出，当前新能源汽车的市场渗透率已经超过 50%，并预计在未来两年内将突破 80% 的大关。这一预测基于多个因素的综合考量，包括新能源汽车价格的持续下降、性能的不断提升以及用户购买力的增强等。随着新能源汽车技术的不断成熟和充电基础设施的日益完善，新能源汽车的普及速度将进一步加快，对传统燃油车的替代效应也将愈发明显。

李斌还透露，蔚来汽车正在为 2025 年至 2026 年的全线产品做准备。F2 工厂已经开始双班生产，并计划在 9 月底到 10 月达到双班生产能力。这一举措将显著提升蔚来汽车的产能，为萤火虫等新品牌的交付提供有力保障。

此外，李斌还强调了换电业务的可持续经营和换电联盟的建立。他表示，换电是蔚来汽车巨大的先发优势，目前已经到了面向全行业开放的时刻。蔚来汽车将与多家企业合作，共同推动换电业务的发展，提升新能源汽车的使用便利性和用户体验。

在谈及市场竞争时，李斌表示，未来几年新能源汽车市场的竞争强度只会增加不会下降。他呼吁全体员工要提升效率、狠抓执行，确保蔚来汽车在未来的竞争中保持领先地位。同时，他也表示蔚来汽车有足够的信心跨越资格赛最惨烈的赛段，赢得决赛的资格。
蔚来汽车第三品牌萤火虫的即将交付以及新能源汽车市场渗透率的快速提升，无疑为整个新能源汽车行业带来了新的发展机遇和挑战。随着新能源汽车技术的不断进步和市场的日益成熟，我们有理由相信新能源汽车的普及速度将进一步加快，为汽车产业的转型升级和可持续发展注入新的动力。"""

In [7]:
documents = [Document(text=text,metadata={'file_name':'test.txt'})]

In [9]:
faiss_index = faiss.IndexFlatL2(512)
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [10]:
base_pipeline = IngestionPipeline(
    transformations=[SentenceSplitter(chunk_size=64,chunk_overlap=12)],
    vector_store=vector_store
)

base_nodes = base_pipeline.run(documents=documents)

In [11]:
node_parser = SentenceWindowNodeParser(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_sentence"
)

pipeline = IngestionPipeline(
    transformations=[node_parser],
    vector_store=vector_store,
)

windowed_nodes = pipeline.run(documents=documents)

In [12]:
query = "蔚来萤火虫主打什么市场？"

In [13]:
base_index = VectorStoreIndex(base_nodes)

base_query_engine = base_index.as_query_engine(
    similarity_top_k=1,
)

In [14]:
base_index = VectorStoreIndex(base_nodes)

base_query_engine = base_index.as_query_engine(
    similarity_top_k=1,
)

base_response = base_query_engine.query(query)

print(base_response)

根据提供的上下文信息，蔚来汽车第三品牌萤火虫即将交付，并且提到新能源汽车市场渗透率的快速提升。虽然没有直接说明蔚来萤火虫主打的具体市场，但可以推测：

**蔚来萤火虫主打的是新能源汽车市场。**

这个推测基于以下几点：
1. **品牌定位**：作为蔚来汽车的第三品牌，萤火虫很可能继承和延续蔚来在新能源汽车领域的定位。
2. **市场趋势**：文中提到新能源汽车市场渗透率的快速提升，暗示萤火虫的推出是为了抓住这一市场机遇。

如果需要更具体的市场细分（如高端市场、大众市场等），则需要更多详细信息。


In [15]:
pprint(base_response.source_nodes[0].node.text)

'蔚来汽车第三品牌萤火虫的即将交付以及新能源汽车市场渗透率的快速提升，无疑为整个新能源汽车行业'


In [16]:
windowed_index = VectorStoreIndex(windowed_nodes)
windowed_query_engine = windowed_index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[
        MetadataReplacementPostProcessor(
            target_metadata_key="window"
            )
        ],
)

windowed_response = windowed_query_engine.query(query)
print(windowed_response)

D:\python311\Lib\site-packages\llama_index\core\schema.py:79: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `OpenAI` - serialized value may not be as expected
  data = handler(self)


蔚来萤火虫主打中低端市场。根据提供的文本信息，蔚来汽车将形成三个品牌矩阵，覆盖从14万元到80万元的广阔市场区间，其中第三品牌萤火虫作为蔚来汽车布局中低端市场的重要棋子，将于2025年正式交付。这一举措旨在丰富蔚来的产品线，并进一步提升其在新能源汽车市场的竞争力。
